In [4]:
from gensim import corpora
from collections import defaultdict
import pandas as pd
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [10]:
documents = pd.read_pickle('gensim/preprocessed_text.pkl')

In [11]:
# remove words that only appear once in all documents
frequency = defaultdict(int)
for text in documents['text']:
    for token in text:
        frequency[token] += 1
        
documents['text'] = [[token for token in text if frequency[token] > 1]
              for text in documents['text']]

In [12]:
documents.head()

,id,language,text
0,16392,English,"[host, review, favorit, favorit, classic, pro,..."
0,18489,English,"[everyon, need, hope, everyon, need, encourag,..."
0,12350,English,"[insur, news, interview, rate, announc, insur,..."
0,12351,English,"[amovetv, crew, talk, video, game, esport, lot..."
0,12352,English,"[pastor, greg, lauri, thirtyminut, daili, radi..."
